In [ ]:
from poitousprint import Portic, Toflit
import json
import networkx as nx
from ipysigma import Sigma

portic_client = Portic()
toflit_client = Toflit()

# this function allows to map a value from a domain of min-max to another
def map_value(value, domain_min, domain_max, range_min, range_max):
    left_span = domain_max - domain_min
    right_span = range_max - range_min

    # Convert the left range into a 0-1 range (float)
    scaled = float(value - domain_min) / float(left_span)

    # Convert the 0-1 range into a value in the right range.
    return range_min + (scaled * right_span)

In [ ]:
# naive toflit18 request
flows = toflit_client.get_flows(
    year=1789,
    customs_region='La Rochelle', 
    params=[
      "product",
      "partner",
      "import",
      "value",
      "line",
      "partner_simplification",
      "customs_office"
	]
)
flows[0]

# Étape 1 : première expérimentation

Cartographie d'un réseau bipartite entre les produits et les partenaires 

In [ ]:
# créer un graphe
Graph = nx.Graph()

# créer des dict pour les deux types de noeuds et les liens
products_uniq = {}
partners_uniq = {}
edges_uniq = {}
# remplir les dicts
for flow in flows:
    partner = flow["partner"]
    product = flow["product"]
    partner_id = "partner_" + partner
    product_id = "product_" + product
    if product_id in products_uniq:
        products_uniq[product_id] = {**products_uniq[product_id], "size": products_uniq[product_id]["size"] + 1}
    else:
       products_uniq[product_id] = {
           "type": "product", 
           "name": product, 
           "color": "rgb(0, 255, 0)",
           "size": 1
       }
    if partner_id in partners_uniq:
        partners_uniq[partner_id] = {**partners_uniq[partner_id], "size": partners_uniq[partner_id]["size"] + 1}
    else:
       partners_uniq[partner_id] = {
           "type": "partner", 
           "name": partner, 
           "color": "rgb(255, 0, 0)",
           "size": 1
       }
    edge_footprint = partner_id + "-" + product_id
    if edge_footprint in edges_uniq:
        edges_uniq[edge_footprint]["weight"] += 1
    else:
        edges_uniq[edge_footprint] = {
            "source": product_id,
            "target": partner_id,
            "weight": 1
        }
# concaténer les deux dicts de noeuds en un seul
all_nodes = partners_uniq
all_nodes.update(products_uniq)
# applatir et formatter les noeuds
nodes = []
for key, node in all_nodes.items():
    nodes.append((key, node))
edges = []

for key, edge in edges_uniq.items():
    edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
    
domain_min_nodes_size = min([node[1]['size'] for node in nodes])
domain_max_nodes_size = max([node[1]['size'] for node in nodes])
range_in_nodes_size = [1, 10]
nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

for node in nodes:
    node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
    node[1]["label"] = node[1]["name"]


Graph.add_nodes_from(nodes)
Graph.add_edges_from(edges)

Sigma(Graph)

# Étape 2 : factoriser la fabrication de réseaux de coocurrence

On fabrique une fonction générique qui permet de fabriquer un réseau à partir :

1. d'une liste de dicts (ex. flux toflit18)
1. d'une liste de deux propriétés à comparer

In [ ]:
def render_coocurrences_graph(data, key_1, key_2, params=None):
    # créer un graphe
    Graph = nx.Graph()

    # créer des dict pour les deux types de noeuds et les liens
    key1_uniq = {}
    key2_uniq = {}
    edges_uniq = {}
    default_params = {
        "color_1": "rgb(0, 255, 0)",
        "color_2": "rgb(255, 0, 0)",
        "node_min_size": 1,
        "node_max_size": 10
    }
    final_params = default_params
    if params is not None :
        final_params = {
            *default_params,
            *params
        }
    
    # remplir les dicts
    for datum in data:
        if key_1 in datum and key_2 in datum:
            value_1 = datum[key_1] if datum[key_1] is not None else "undefined"
            value_2 = datum[key_2] if datum[key_2] is not None else "undefined"
            value_1_id = key_1 + "_" + value_1
            value_2_id = key_2 + "_" + value_2
            if value_1_id in key1_uniq:
                key1_uniq[value_1_id] = {**key1_uniq[value_1_id], "size": key1_uniq[value_1_id]["size"] + 1}
            else:
               key1_uniq[value_1_id] = {
                   "type": key_1, 
                   "name": value_1, 
                   "color": final_params["color_1"],
                   "size": 1
               }
            if value_2_id in key2_uniq:
                key2_uniq[value_2_id] = {**key2_uniq[value_2_id], "size": key2_uniq[value_2_id]["size"] + 1}
            else:
               key2_uniq[value_2_id] = {
                   "type": key_2, 
                   "name": value_2, 
                   "color": final_params["color_2"],
                   "size": 1
               }
            edge_footprint = value_1_id + "-" + value_2_id
            if edge_footprint in edges_uniq:
                edges_uniq[edge_footprint]["weight"] += 1
            else:
                edges_uniq[edge_footprint] = {
                    "source": value_1_id,
                    "target": value_2_id,
                    "weight": 1
                }
    # concaténer les deux dicts de noeuds en un seul
    all_nodes = key1_uniq
    all_nodes.update(key2_uniq)
    # applatir et formatter les noeuds
    nodes = []
    for key, node in all_nodes.items():
        nodes.append((key, node))
    edges = []

    for key, edge in edges_uniq.items():
        edges.append((edge["source"], edge["target"], {"weight": edge["weight"]}))
        
    # ajuster la taille des noeuds en fonction d'un min et d'un max donnés
    domain_min_nodes_size = min([node[1]['size'] for node in nodes])
    domain_max_nodes_size = max([node[1]['size'] for node in nodes])
    range_in_nodes_size = [final_params["node_min_size"], final_params["node_max_size"]]
    nodes_size_mapping_params = [domain_min_nodes_size, domain_max_nodes_size, *range_in_nodes_size]

    for node in nodes:
        node[1]["size"] = map_value(node[1]["size"], *nodes_size_mapping_params)
        node[1]["label"] = node[1]["name"]


    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(edges)

    return Sigma(Graph, start_layout=True)

In [ ]:
render_coocurrences_graph(flows, "product", "partner_simplification")

In [ ]:
render_coocurrences_graph(flows, "partner", "partner_simplification")

# Étape 3 : test avec PORTIC


In [ ]:
flows = portic_client.get_flows(year=1789)

In [ ]:
flows[0]

In [ ]:
len(flows)

In [ ]:
render_coocurrences_graph(flows, "departure_fr", "destination_fr")

In [ ]:
render_coocurrences_graph(flows, "homeport_toponyme_fr", "destination_fr")

# Étape 4 : test des croisements

In [ ]:
def resolve_flow_partner(flow):
    res = flow.copy()
    partner = flow['destination_partner_balance_1789']
    if flow is None and 'destination_partner_balance_supp_1789' in flow:
        partner = flow['destination_partner_balance_supp_1789']
    if flow is None:
        partner = 'non attribué'
    res['partner'] = partner
    return res
    
flows_with_toflit18_partner = [resolve_flow_partner(flow) for flow in flows if flow["source_subset"] == "Poitou_1789"]

render_coocurrences_graph(flows_with_toflit18_partner, "departure", "partner")

In [ ]:
unique_partners = set([t["partner"] for t in flows_with_toflit18_partner])
unique_partners

In [ ]:
# test sans la france
flows_with_toflit18_partner=[flow for flow in flows_with_toflit18_partner if flow["partner"] != "France" and flow["partner"] is not None]
render_coocurrences_graph(flows_with_toflit18_partner, "departure", "partner")